In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

# List collections
print(client.get_collections())

# Check info collection
info = client.get_collection("chat_history")
print(info)


collections=[CollectionDescription(name='rag_knowledge'), CollectionDescription(name='chat_history')]
status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=2 segments_count=6 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_thr

In [5]:
from qdrant_client import QdrantClient
from pprint import pprint

client = QdrantClient(host="localhost", port=6333)

def inspect_qdrant():
    collections = client.get_collections().collections
    print(f"Found {len(collections)} collections:")
    
    for col in collections:
        print("\n" + "="*40)
        print(f"Collection name: {col.name}")
        try:
            info = client.get_collection(col.name)
            print(f"Points count: {info.points_count}")
            vectors = info.config.params.vectors
            if isinstance(vectors, dict):
                print("Vector names & sizes:")
                for name, vcfg in vectors.items():
                    print(f"  - {name}: size={vcfg.size}, distance={vcfg.distance}")
            else:
                print(f"Vector config: size={vectors.size}, distance={vectors.distance}")
        except Exception as e:
            print("Could not fetch collection info:", e)
        
        print("Sample points (up to 10):")
        try:
            resp, _ = client.scroll(col.name, limit=10, with_payload=True)
            points = getattr(resp, "result", resp)
            for p in points:
                vector_names = list(p.vector.keys()) if isinstance(p.vector, dict) else ["default"]
                print(f"  id={p.id}, vector_names={vector_names}, payload={p.payload}")
        except Exception as e:
            print("Could not fetch points:", e)

if __name__ == "__main__":
    inspect_qdrant()


Found 1 collections:

Collection name: collection
Points count: 0
Vector config: size=384, distance=Cosine
Sample points (up to 10):


In [19]:
from qdrant_client import QdrantClient

# Kết nối Qdrant (ví dụ chạy local)
client = QdrantClient("http://localhost:6333")

collection_name = "collection"

# Lấy thông tin collection
info = client.get_collection(collection_name)
print("Tổng số vector:", info.vectors_count)


Tổng số vector: None


In [7]:
import numpy as np

# Giả sử bạn có embedding vector test 
dummy_vector = np.random.rand(384).tolist()

result = client.search(
    collection_name=collection_name,
    query_vector=dummy_vector,
    limit=3
)

print("Kết quả tìm được:", result)


Kết quả tìm được: []


C:\Users\admin\AppData\Local\Temp\ipykernel_14208\2745754574.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  result = client.search(


In [2]:
import qdrant_client

def check_collections():
    client = qdrant_client.QdrantClient(host="localhost", port=6333)

    collections = client.get_collections()
    print("📂 Danh sách collections trong Qdrant:")
    for coll in collections.collections:
        name = coll.name

        # Dùng count để chắc chắn có số vector
        count = client.count(collection_name=name, exact=True).count
        print(f" - {name}: {count} vectors")

    if not collections.collections:
        print("⚠️ Không có collection nào trong Qdrant (rỗng).")

if __name__ == "__main__":
    check_collections()


📂 Danh sách collections trong Qdrant:
 - collection: 0 vectors


In [14]:
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost:6333")

# Xem các collection hiện có
print(client.get_collections())

# Nếu có collection "documents", xem thử số lượng points
print(client.count(collection_name="collection", exact=True))

# Lấy vài point mẫu
points = client.scroll(collection_name="collection", limit=5)
print(points)


collections=[CollectionDescription(name='collection')]
count=1
([Record(id=351531, payload={'source': 'app'}, vector=None, shard_key=None, order_value=None)], None)


In [4]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://cafef.vn"


def get_url(page: int) -> str:
    return f"{BASE_URL}/doc-nhanh.chn" if page == 1 else f"{BASE_URL}/doc-nhanh/trang-{page}.chn"


def get_article_content(link: str) -> str:
    """Vào trang chi tiết và lấy toàn bộ nội dung"""
    resp = requests.get(link, timeout=10)
    resp.encoding = "utf-8"
    soup = BeautifulSoup(resp.text, "html.parser")

    content_div = soup.select_one("div.detail-content")
    if not content_div:
        return ""

    # Ghép các đoạn văn lại
    paragraphs = [p.get_text(strip=True) for p in content_div.find_all("p") if p.get_text(strip=True)]
    return "\n".join(paragraphs)


def crawl_cafef(max_pages: int = 1):
    articles = []
    for page in range(1, max_pages + 1):
        url = get_url(page)
        resp = requests.get(url, timeout=10)
        resp.encoding = "utf-8"
        soup = BeautifulSoup(resp.text, "html.parser")

        blocks = soup.select("div.nv-text-cont")
        for block in blocks:
            link_tag = block.select_one("a.news-title")
            if not link_tag:
                continue

            href = link_tag["href"]
            article_id = href.split("-")[-1].replace(".chn", "")
            title = link_tag.get("title") or link_tag.text.strip()
            link = BASE_URL + href

            details = block.find_next_sibling("div", class_="nv-details")
            short_content = details.select_one("div.abs").text.strip() if details else ""
            time_tag = details.select_one("span.time")
            time_text = time_tag.get_text(strip=True) if time_tag else ""

            # Lấy toàn bộ nội dung trong link
            full_content = get_article_content(link)

            articles.append({
                "id": article_id,
                "title": title,
                "time": time_text,
                "summary": short_content,
                "link": link,
                "content": full_content
            })
    return articles


if __name__ == "__main__":
    data = crawl_cafef(max_pages=2)
    for item in data[:3]:
        print(item)


{'id': '188250924111917324', 'title': 'TP HCM: Tin vui cho người dân ở chung cư cũ', 'time': '(24/09/2025 12:41)', 'summary': 'Chung cư xây dựng trước năm 1994  sẽ được hỗ trợ cải tạo, xây dựng lại; TP HCM cũng kiểm định lại chất lượng hàng trăm chung cư cũ.', 'link': 'https://cafef.vn/tp-hcm-tin-vui-cho-nguoi-dan-o-chung-cu-cu-188250924111917324.chn', 'content': 'Chủ tịch UBND TP HCM vừa chỉ đạo triển khai Nghị quyết số 17/2025 của HĐND TP HCM quy định cơ chế ưu đãi, hỗ trợ đối với chủ đầu tư khi tham gia cải tạo, xây dựng lại nhà chung cư trên địa bàn thành phố.\nTP HCM triển khai nhiều nội dung hỗ trợ chung cư cũ xây dựng trước năm 1994.\nTheo đó, Chủ tịch UBND TP HCM giao Sở Tài chính chủ trì, hướng dẫn Sở Xây dựng là cơ quan được bố trí dự toán kinh phí từ nguồn vốn chi thường xuyên ngân sách cấp thành phố để thực hiện cơ chế theo Nghị quyết 17/2025.\nUBND 42 phường có chung cư xây dựng trước năm 1994 trên địa bàn thông tin đến tổ chức, cá nhân tại vị trí, khu vực có chung cư thuộ

In [ ]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://cafef.vn"


def get_url(page: int) -> str:
    """Sinh URL cho từng trang tin nhanh."""
    return f"{BASE_URL}/doc-nhanh.chn" if page == 1 else f"{BASE_URL}/doc-nhanh/trang-{page}.chn"


def get_article_content(link: str) -> str:
    """Vào trang chi tiết và lấy toàn bộ nội dung bài viết."""
    try:
        resp = requests.get(link, timeout=10)
        resp.encoding = "utf-8"
    except Exception as e:
        print(f"[Crawler] Lỗi khi request {link}: {e}")
        return ""

    if resp.status_code != 200:
        print(f"[Crawler] Lỗi {resp.status_code} khi truy cập {link}")
        return ""

    soup = BeautifulSoup(resp.text, "html.parser")

    content_div = soup.select_one("div.detail-content")
    if not content_div:
        return ""

    # Ghép các đoạn văn lại
    paragraphs = [
        p.get_text(strip=True)
        for p in content_div.find_all("p")
        if p.get_text(strip=True)
    ]
    return "\n".join(paragraphs)


def crawl_cafef(max_pages: int = 1):
    """Crawl tin nhanh trên CafeF, trả về list dict bài viết."""
    articles = []
    for page in range(1, max_pages + 1):
        url = get_url(page)
        try:
            resp = requests.get(url, timeout=10)
            resp.encoding = "utf-8"
        except Exception as e:
            print(f"[Crawler] Lỗi khi request {url}: {e}")
            continue

        if resp.status_code != 200:
            print(f"[Crawler] Lỗi {resp.status_code} khi truy cập {url}")
            continue

        soup = BeautifulSoup(resp.text, "html.parser")

        # Mỗi bài trong block fast-news
        items = soup.select("div.foreverblock.list-fast-news div.item")
        for item in items:
            nv_text = item.select_one("div.nv-text-cont")
            if not nv_text:
                continue

            link_tag = nv_text.select_one("a.news-title")
            if not link_tag:
                continue

            href = link_tag["href"]
            article_id = item.get("data-id") or href.split("-")[-1].replace(".chn", "")
            title = link_tag.get("title") or link_tag.text.strip()
            link = BASE_URL + href if href.startswith("/") else href

            details = item.select_one("div.nv-details")
            summary = (
                details.select_one("div.abs").get_text(strip=True)
                if details and details.select_one("div.abs")
                else ""
            )
            time_text = (
                details.select_one("span.time").get_text(strip=True)
                if details and details.select_one("span.time")
                else ""
            )

            # Crawl nội dung chi tiết
            full_content = get_article_content(link)

            articles.append(
                {
                    "id": article_id,
                    "title": title,
                    "time": time_text,
                    "summary": summary,
                    "link": link,
                    "content": full_content,
                }
            )

    print(f"[Crawler] Crawled {len(articles)} articles")
    return articles


if __name__ == "__main__":
    data = crawl_cafef(max_pages=2)
    for item in data[:2]:
        print(item)


In [7]:
from qdrant_client import QdrantClient
client = QdrantClient()
print(client.get_collections())


collections=[CollectionDescription(name='collection')]


In [6]:
from qdrant_client import QdrantClient, models
import uuid
from modules.utils.services import embedder_services   # dùng MiniLM encode_dense


# 1. Kết nối Qdrant
qdrant = QdrantClient(host="localhost", port=6333)
collection_name = "test_hybrid"

# 2. Tạo collection (nếu chưa có)
qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config={
        "dense_vector": models.VectorParams(
            size=384,
            distance=models.Distance.COSINE
        ),
        "binary": models.VectorParams(
            size=384,
            distance=models.Distance.DOT,
            on_disk=True
        ),
    },
    sparse_vectors_config={
        "sparse_vector": models.SparseVectorParams()
    }
)

# 3. Thêm vài documents
docs = [
    {"title": "Python cơ bản", "content": "Python là ngôn ngữ lập trình dễ học và mạnh mẽ."},
    {"title": "Học máy", "content": "Machine Learning giúp máy tính học từ dữ liệu."},
    {"title": "Bóng đá", "content": "Messi là cầu thủ xuất sắc trong lịch sử bóng đá."}
]

for d in docs:
    vec = embedder_services.encode_dense(d["content"])[0]
    qdrant.upsert(
        collection_name=collection_name,
        points=[
            models.PointStruct(
                id=str(uuid.uuid4()),
                vector={"dense_vector": vec, "binary": vec},  # insert cả dense + binary
                payload=d
            )
        ]
    )

print("✅ Inserted documents")

# 4. Search hybrid (dense + bm25 + binary)
query_text = "ngôn ngữ lập trình Python"
dense_vec = embedder_services.encode_dense(query_text)[0]

results = qdrant.query_points(
    collection_name=collection_name,
    prefetch=[
                models.Prefetch(
                    query=dense_vec,
                    using="dense_vector"
                ),
                models.Prefetch(query="ngôn ngữ lập trình Python", using="bm25"),
                models.Prefetch(
                    query=dense_vec,
                    using="binary"      #quantized
                )
    ],
    query=models.FusionQuery(fusion=models.Fusion.RRF),
    limit=3,
    with_payload=True,
)

# 5. In kết quả
print("🔎 Search results for:", query_text)
for r in results:
    print(f"- {r.payload.get('title')} | score={r.score:.3f}")


C:\Users\admin\AppData\Local\Temp\ipykernel_4796\2529383509.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


✅ Inserted documents


UnexpectedResponse: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Format error in JSON body: Expected some form of vector, id, or a type of query at line 1 column 15724"},"time":0.0}'

In [17]:
from qdrant_client import QdrantClient

def list_vectors_in_qdrant(collection_name: str = "my_collection", host: str = "localhost", port: int = 6333, limit: int = 100):
    """
    Liệt kê tất cả vectors trong collection và in ID + payload + độ dài vector.
    """
    client = QdrantClient(host=host, port=port)

    try:
        points, next_page = client.scroll(
            collection_name=collection_name,
            limit=limit
        )

        if not points:
            print(f"Collection '{collection_name}' hiện chưa có vector nào.")
        else:
            print(f"Collection '{collection_name}' có {len(points)} vector (lấy tối đa {limit}):")
            for point in points:
                print(f"ID: {point.id}, payload: {point.payload}, vector length: {len(point.vector) if point.vector else 0}")

            # nếu còn next_page thì nghĩa là collection còn nhiều vector nữa
            if next_page:
                print("⚠️ Collection còn vector khác, hãy tăng limit hoặc tiếp tục scroll với offset.")
    except Exception as e:
        print("Lỗi khi truy vấn Qdrant:", e)

# Test
list_vectors_in_qdrant(collection_name="collection")


Collection 'collection' có 12 vector (lấy tối đa 100):
ID: 86651, payload: {'source': 'app'}, vector length: 0
ID: 175468, payload: {'source': 'app'}, vector length: 0
ID: 190315, payload: {'source': 'app'}, vector length: 0
ID: 236817, payload: {'source': 'app'}, vector length: 0
ID: 318901, payload: {'source': 'app'}, vector length: 0
ID: 351531, payload: {'source': 'app'}, vector length: 0
ID: 462858, payload: {'source': 'app'}, vector length: 0
ID: 534956, payload: {'source': 'app'}, vector length: 0
ID: 586207, payload: {'source': 'app'}, vector length: 0
ID: 603741, payload: {'source': 'app'}, vector length: 0
ID: 798773, payload: {'source': 'app'}, vector length: 0
ID: 845293, payload: {'source': 'app'}, vector length: 0


In [1]:
import requests
r = requests.get("https://cafef.vn/doc-nhanh.chn", headers={"User-Agent": "Mozilla/5.0"})
print(r.text[:1000])


<!DOCTYPE html>
<html lang="vi">
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
    <title>Xem nhanh tin mới cập nhật</title>
    <meta name="description" content="Xem nhanh các tin mới cập nhật trên CafeF.vn">
    <meta name="keywords" content="thị trường,chứng khoán, cập nhật ,nhà đầu tư,đầu đủ, lịch sự kiện,tin tuc chung khoan, tin chứng khoán, chỉ số HNX index, sàn hà nội, sàn sài gòn, hose, thông tin thị trường chứng khoán, lịch sự kiện, chứng khoán việt nam, tin chứng khoán việt nam, hose,vn index, tin vnindex">
    <meta name="news_keywords" content="thị trường,chứng khoán, cập nhật ,nhà đầu tư,đầu đủ, lịch sự kiện,tin tuc chung khoan, tin chứng khoán, chỉ số HNX index, sàn hà nội, sàn sài gòn, hose, thông tin thị trường chứng khoán, lịch sự kiện, chứng khoán việt nam, tin chứng khoán việt nam, hose,vn index, tin vnindex">
    <meta property="og:title" content="">
    <meta property="og:description" content="Xem nhanh các tin mới cập nhật trên C

In [ ]:
from modules.utils.qdrant_utils import add_doc, update_doc, delete_doc, get_doc, search_hybrid

# Thêm tài liệu
pid = add_doc(
    collection_name="test_hybrid",
    payload={"title": "Python", "content": "Python là ngôn ngữ lập trình."}
)

# Cập nhật payload
update_doc("test_hybrid", pid, {"category": "programming"})

# Lấy payload
print(get_doc("test_hybrid", pid))

# Hybrid search
print(search_hybrid("test_hybrid", [0.1] * 384, "ngôn ngữ lập trình Python"))

# Xóa document
delete_doc("test_hybrid", pid)


In [3]:
from modules.utils.services import qdrant_services
print(qdrant_services.client.get_collection("test_hybrid"))


status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=1 segments_count=6 config=CollectionConfig(params=CollectionParams(vectors={'binary': VectorParams(size=384, distance=<Distance.DOT: 'Dot'>, hnsw_config=None, quantization_config=None, on_disk=True, datatype=None, multivector_config=None), 'dense_vector': VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors={'sparse_vector': SparseVectorParams(index=None, modifier=None)}), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, 

In [2]:
from modules.utils.qdrant_utils import add_doc, get_doc, search_hybrid, delete_doc
from modules.utils.services import embedder_services
corpus = [
    "Python là ngôn ngữ lập trình.",
    "Ngôn ngữ C++ hiệu suất cao.",
    "Java phổ biến cho ứng dụng doanh nghiệp."
]
embedder_services.fit_bm25(corpus)
pid = add_doc("test_hybrid", [0.1]*384, None, {"title": "Python", "content": "Python là ngôn ngữ lập trình."})
print(get_doc("test_hybrid", pid))
print(search_hybrid("test_hybrid", "ngôn ngữ lập trình Python"))
# delete_doc("test_hybrid", pid)


{'title': 'Python', 'content': 'Python là ngôn ngữ lập trình.'}
[{'id': '721c345a-5277-4da4-8b8c-c4eefdb02824', 'score': 1.0, 'payload': {'title': 'Python', 'content': 'Python là ngôn ngữ lập trình.'}}]


In [4]:
import rank_bm25
print(rank_bm25.__file__)


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\rank_bm25.py


In [3]:
import sys
print(sys.executable)  # đường dẫn python đang chạy
!{sys.executable} -m pip install rank-bm25


c:\Users\admin\AppData\Local\Programs\Python\Python311\python.exe
  Using cached rank_bm25-0.2.2-py3-none-any.whl.metadata (3.2 kB)
Using cached rank_bm25-0.2.2-py3-none-any.whl (8.6 kB)
